<a href="https://colab.research.google.com/github/CaoHaiNam/Content-base-Recommendation-system/blob/master/final_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# to import .py module
import sys
import os
py_file_location = "/content/drive/My Drive/Norm_add_based_recommendation"
sys.path.append(os.path.abspath(py_file_location))

In [2]:
import codecs
import json
import preprocess
import word2vec
import numpy as np
import theano.tensor as T
import keras
from keras import backend as K
from keras import initializers
from keras.models import Sequential, Model, load_model, save_model
from keras.layers.core import Dense, Lambda, Activation
from keras.layers import Embedding, Input, Dense, merge, Reshape, Concatenate, Flatten, SimpleRNN
from keras.optimizers import Adagrad, Adam, SGD, RMSprop
from keras.regularizers import l2
from time import time
import multiprocessing as mp
import sys
import math
import argparse

In [8]:
keras.__version__

'2.4.3'

In [3]:
def get_reshape_model(Embedding_dim):
    model = keras.Sequential()
    model.add(keras.layers.Reshape((Embedding_dim,), input_shape=(1,Embedding_dim)))
    return model

In [9]:
def get_Neu_model(cell = 41,
                  features = 300,
                  latent_dim = 20,
                  embedding_dim = 20, 
                  num_street = 1881, 
                  num_ward = 425,
                  num_district = 286,
                  num_city = 62,
                  layers = [20, 10]):
    # reshape model
    reshape_model = get_reshape_model(embedding_dim)
    num_layers = len(layers)

    raw_input           = Input(shape=(cell, features), dtype=np.float32, name='raw_input')
    norm_input          = Input(shape=(cell, features), dtype=np.float32, name='norm_input')
    norm_street_input   = Input(shape=(1,), dtype=np.float32, name='norm_street')
    norm_ward_input     = Input(shape=(1,), dtype=np.float32, name='norm_ward')
    norm_district_input = Input(shape=(1,), dtype=np.float32, name='norm_district')
    norm_city_input     = Input(shape=(1,), dtype=np.float32, name='norm_city')

    # GMF_vector
    raw_lstm  = keras.layers.LSTM(latent_dim + 4*embedding_dim)
    norm_lstm = keras.layers.LSTM(latent_dim)
    # plus 1 in case value of this field is None
    Embedding_street   = Embedding(input_dim=num_street,   output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)
    Embedding_ward     = Embedding(input_dim=num_ward,     output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)
    Embedding_district = Embedding(input_dim=num_district, output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)
    Embedding_city     = Embedding(input_dim=num_city,     output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)

    raw_latent = raw_lstm(raw_input)
    norm_latent = norm_lstm(norm_input)
    street_latent   = reshape_model(Embedding_street(norm_street_input))
    ward_latent     = reshape_model(Embedding_ward(norm_street_input))
    district_latent = reshape_model(Embedding_district(norm_street_input))
    city_latent     = reshape_model(Embedding_city(norm_street_input))
    norm_latent = keras.layers.Concatenate()([norm_latent, street_latent, ward_latent, district_latent, city_latent])

    # Element-wise product of raw and norm embeddings
    GMF_vector = keras.layers.Multiply()([raw_latent, norm_latent])

    # MLP_vector
    raw_lstm  = keras.layers.LSTM(latent_dim)
    norm_lstm = keras.layers.LSTM(latent_dim)
    # plus 1 in case value of this field is None
    Embedding_street   = Embedding(input_dim=num_street,   output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)
    Embedding_ward     = Embedding(input_dim=num_ward,     output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)
    Embedding_district = Embedding(input_dim=num_district, output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)
    Embedding_city     = Embedding(input_dim=num_city,     output_dim=embedding_dim, embeddings_initializer=keras.initializers.random_uniform, input_length=1)

    raw_latent  = raw_lstm(raw_input)
    norm_latent = norm_lstm(norm_input)
    street_latent   = reshape_model(Embedding_street(norm_street_input))
    ward_latent     = reshape_model(Embedding_ward(norm_street_input))
    district_latent = reshape_model(Embedding_district(norm_street_input))
    city_latent     = reshape_model(Embedding_city(norm_street_input))

    # The 0-th layer is the concatenation of embedding layers
    MLP_vector = keras.layers.Concatenate()([raw_latent, norm_latent, street_latent, ward_latent, district_latent, city_latent]) 
    
    # MLP layer
    for idx in range(num_layers):
      layer = Dense(units=layers[idx], activation='relu') 
      MLP_vector = layer(MLP_vector)

    # final prediction vector
    predict_vector = keras.layers.Concatenate()([GMF_vector, MLP_vector])

    # Final prediction layer
    prediction = Dense(1, activation='sigmoid', name = "prediction")(predict_vector)

    model = keras.models.Model(inputs=[raw_input, norm_input, norm_street_input, norm_ward_input, norm_district_input, norm_city_input], outputs=prediction)
    
    return model

In [10]:
final_model = get_Neu_model()
final_model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
norm_street (InputLayer)        [(None, 1)]          0                                            
__________________________________________________________________________________________________
raw_input (InputLayer)          [(None, 41, 300)]    0                                            
__________________________________________________________________________________________________
norm_input (InputLayer)         [(None, 41, 300)]    0                                            
__________________________________________________________________________________________________
embedding_16 (Embedding)        (None, 1, 20)        37620       norm_street[0][0]                
_______________________________________________________________________________________